In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# Summarize dialogue without Prompt Engineering

In [2]:
huggingface_dataset= 'knkarthick/dialogsum'
dataset=load_dataset(huggingface_dataset)

Found cached dataset csv (C:/Users/Hashif/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
example_indices= [40,200]
dash_line='-'.join(''for x in range(100))
for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('Base line human summary:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

 



---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Base line human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

In [4]:
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
tokeniser= AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [6]:
sentence ='What time is it, Tom?'
sentence_encoded= tokeniser(sentence,return_tensors='pt')
sentence_decoded= tokeniser.decode(sentence_encoded['input_ids'][0],skip_special_tokens=True)
print('ENCODED SENTENCES:')
print(sentence_encoded['input_ids'][0])
print('\nDecoded sentences:')
print(sentence_decoded)

ENCODED SENTENCES:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

Decoded sentences:
What time is it, Tom?


In [28]:
for i, index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    inputs= tokeniser(dialogue,return_tensors='pt')
    output=tokeniser.decode(model.generate(
        inputs['input_ids'],
        max_new_tokens=50,)[0],
        skip_special_tokens=True)
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print(f'Input dialogue:\n{dialogue}')
    print(dash_line)
    print(f'Baseline human summary:\n{summary}')
    print(dash_line)   
    print(f'Model Gneration-Without prompt engineering:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Gneration- ZERO SHOT:
Person1: It's ten to nine.

-------------------------------------------------

# Summarize Dialogue with Prompt Engineering
### 1. ZERO SHOT INFERENCE

In [7]:
for i, index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
   
    prompt=f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    inputs= tokeniser(prompt,return_tensors='pt')
    output=tokeniser.decode(model.generate(
        inputs['input_ids'],
        max_new_tokens=50,)[0],
        skip_special_tokens=True)
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print(f'Input Prompt:\n{prompt}')
    print(dash_line)
    print(f'Baseline human summary:\n{summary}')
    print(dash_line)   
    print(f'Model Gneration- ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt:

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Gneration- ZERO SHOT:
The train is about to le

In [8]:
for i, index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
   
    prompt=f"""
Dialogue:
{dialogue}

What was  going on?
    """

    inputs= tokeniser(prompt,return_tensors='pt')
    output=tokeniser.decode(model.generate(
        inputs['input_ids'],
        max_new_tokens=50,)[0],
        skip_special_tokens=True)
    print('Example',i+1)
    print(dash_line)
    print(f'Input Prompt:\n{prompt}')
    print(dash_line)
    print(f'Baseline human summary:\n{summary}')
    print(dash_line)   
    print(f'Model Gneration- ZERO SHOT:\n{output}\n')

Example 1
---------------------------------------------------------------------------------------------------
Input Prompt:

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was  going on?
    
---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Gneration- ZERO SHOT:
Tom is late for the train.

Example 2
---------------------------------------------------------------------------------------------------
Inpu

### 2. ONE SHOT INFERENCE

In [11]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt=''
    for index in example_indices_full:
        dialogue=dataset['test'][index]['dialogue']
        summary=dataset['test'][index]['summary']
        
        prompt+=f"""
Dialogue:
{dialogue}

What was  going on?
{summary}

"""
    dialogue=dataset['test'][example_index_to_summarize]['dialogue']

    prompt+=f"""
Dialogue:
{dialogue}

What was  going on?
"""
    return prompt
        

In [12]:
example_indices_full=[40]
example_index_to_summarize=200
one_shot_prompt= make_prompt([40],200)
print(one_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was  going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need

In [16]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs= tokeniser(one_shot_prompt,return_tensors='pt')
output=tokeniser.decode(model.generate(
        inputs['input_ids'],
        max_new_tokens=50,)[0],
        skip_special_tokens=True)
print(dash_line)
print(f'Baseline human summary:\n{summary}')
print(dash_line)   
print(f'Model Gneration- ONE SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model Gneration- ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.



### 3. FEW SHOT INFERENCE

In [14]:
example_indices_full=[40,80,120]
example_index_to_summarize=200
few_shot_prompt= make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was  going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. Ma

In [17]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs= tokeniser(few_shot_prompt,return_tensors='pt')
output=tokeniser.decode(model.generate(
        inputs['input_ids'],
        max_new_tokens=50,)[0],
        skip_special_tokens=True)
print(dash_line)
print(f'Baseline human summary:\n{summary}')
print(dash_line)   
print(f'Model Gneration- ONE SHOT:\n{output}\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model Gneration- ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.



# GENERATIVE CONFIGURATION PARAMETERS FOR INFERENCE

In [27]:
#generation_config= GenerationConfig(max_new_tokens=50)
generation_config= GenerationConfig(max_new_tokens=50,do_sample=True, temperature=0.5)


inputs= tokeniser(few_shot_prompt,return_tensors='pt')
output=tokeniser.decode(model.generate(
        inputs['input_ids'],
        generation_config=generation_config,)[0],
        skip_special_tokens=True)
print(dash_line)
print(f'Baseline human summary:\n{summary}')
print(dash_line)   
print(f'Model Gneration- few SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Baseline human summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model Gneration- few SHOT:
#Person1 offers advice on how to upgrade a computer. #Person2 suggests adding a painting program to the software.

